<a href="https://colab.research.google.com/github/3veryDay/Project_Festival/blob/main/BLOG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# ✅ 필수 라이브러리 설치
!pip install requests playwright
!playwright install chromium
!playwright install
!pip install boto3

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 22.5 MB/s eta 0:00:00
164.9 MiB [] 0% 10.9s164.9 MiB [] 0% 48.1s164.9 MiB [] 0% 24.5s164.9 MiB [] 0% 16.0s164.9 MiB [] 0% 11.2s164.9 MiB [] 1% 5.9s164.9 MiB [] 2% 4.7s164.9 MiB [] 2% 4.0s164.9 MiB [] 3% 3.6s164.9 MiB [] 4% 3.2s164.9 MiB [] 5% 2.9s164.9 MiB [] 6% 2.7s164.9 MiB [] 6% 2.8s164.9 MiB [] 6% 3.0s164.9 MiB [] 6% 3.2s164.9 MiB [] 7% 2.9s164.9 MiB [] 8% 3.1s164.9 MiB [] 8% 2.9s164.9 MiB [] 9% 2.8s164.9 MiB [] 10% 2.9s164.9 MiB [] 10% 2.8s164.9 MiB [] 11% 2.8s164.9 MiB [] 11% 2.9s164.9 MiB [] 12% 2.8s164.9 MiB [] 13% 2.7s164.9 MiB [] 14% 2.5s164.9 MiB [] 15% 2.5s164.9 MiB [] 16% 2.4s164.9 MiB [] 16% 2.3s164.9 MiB [] 17% 2.3s164.9 MiB [] 18% 2.3s164.9 MiB [] 20% 2.1s164.9 MiB [] 21% 2.0s164.9 MiB [] 22% 1.9s164.9 MiB [] 23% 2.0s164.9 MiB [] 23% 2.1s164.9 MiB [] 24% 2.1s164.9 MiB [] 25% 1.9s164.9 MiB [] 26% 1.9s164.9 MiB [] 27% 1.8s164.9 MiB [] 28% 1.8s164.9 MiB [] 29% 1.8s164.9 MiB [] 30% 1.7s164.9 

In [2]:
import requests
import os
import time
import re
import uuid
import asyncio
import json
from playwright.async_api import async_playwright

# 네이버 API 인증 정보
client_id = "i3d2KviD1ZbHgTcLqpRo"
client_secret = "i4Go7aos27"

def get_article_list_from_naver_api(name, max_results=1000):
    url = "https://openapi.naver.com/v1/search/blog.json"
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    query = f'"{name}"'
    print(f"🔍 {query}로 검색 시작")

    results = []
    for start in range(1, max_results + 1, 100):
        params = {
            "query": query,
            "display": 100,
            "start": start
        }

        print(f"📦 {start}번부터 불러오는 중...")

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            items = data.get("items", [])
            results.extend(items)

            if len(items) < 100:
                print("더 이상 결과가 없습니다.")
                break
            time.sleep(1)

    print(f"\n✅ 총 {len(results)}개 게시글 수집 완료!")
    return results

In [3]:
async def get_blog_article_async(num, url, festival_name):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        try:
            await page.goto(url)
            await page.wait_for_timeout(3000)

            frame = page.frame(name="mainFrame")
            if not frame:
                print(f"🚫 [{num + 1}] iframe(mainFrame) 못 찾음: {url}")
                return None

            # 제목
            try:
                title = await frame.locator('p.se-text-paragraph span').first.inner_text()
            except:
                title = "(제목 없음)"

            # 글쓴이
            try:
                writer = await frame.locator('a.link.pcol2').first.inner_text()
            except:
                writer = "(글쓴이 없음)"

            # 날짜
            try:
                date = await frame.locator('span.se_publishDate.pcol2').first.inner_text()
            except:
                date = "(날짜 없음)"

            # 공감수
            try:
                like_raw = await frame.locator('em.u_cnt._count').first.inner_text()
                like_count = int(re.sub(r'[^0-9]', '', like_raw))
            except:
                like_count = 0

            # 본문
            try:
                content_blocks = await frame.locator('div.se-component-content').all()
                content_list = [await block.inner_text() for block in content_blocks]
                content = "\n".join(content_list)
            except:
                content = "(본문 없음)"

            result = {
                'reviewID': str(uuid.uuid4()),
                'index': num,
                'url': url,
                '글 제목': title,
                '글쓴이': writer,
                '축제명': festival_name,
                '날짜': date,
                '공감수': like_count,
                '본문 글': content
            }

            print(f"✅ [{num + 1}] 크롤링 성공")
            return result

        except Exception as e:
            print(f"🚫 [{num + 1}] 크롤링 실패: {url}")
            print(f"에러: {e}")
            return None

        finally:
            await browser.close()

In [4]:
def filter_relevant_posts(posts, keyword):
    # 공백 무시하는 정규표현식 패턴 생성
    pattern = r"".join([f"{char}\s*" for char in keyword])
    keyword_regex = re.compile(pattern, re.IGNORECASE)

    filtered = []
    for post in posts:
        title = re.sub(r"<.*?>", "", post["title"])
        desc = re.sub(r"<.*?>", "", post["description"])
        combined = title + " " + desc
        if re.search(keyword_regex, combined):
            filtered.append(post)
    print(f"✅ 필터링 후 남은 글 수: {len(filtered)}")
    return filtered

In [ ]:
async def main():
    # 🎯 크롤링할 축제명 입력
        name = "신촌글로벌대학문화축제"

    # 📍 1. 블로그 글 리스트 가져오기
        raw_posts = get_article_list_from_naver_api(name)

    # 제목/요약에 정확히 '더북데이' 들어간 것만 필터링
        posts = filter_relevant_posts(raw_posts, name)


    # 📍 2. 상세 크롤링 시작
        crawled_results = []
        print("\n🔍 블로그 게시글 상세 내용 크롤링 시작...\n")
        async with async_playwright() as p:
        # 브라우저 설정
          browser = await p.chromium.launch(headless=True)
          context = await browser.new_context()
          page = await context.new_page()

          # 3. JSON 파일로 저장 (축제명 기반 파일명) path 설정하기
          safe_name = name.replace(" ", "_")  # 공백을 밑줄로 바꾸기
          json_filename = f"results/{safe_name}.json"
          file_name = f"{safe_name}_블로그_크롤링결과.json"
          path = '/content/drive/MyDrive/JSON/BLOG/' + file_name




          for i, post in enumerate(posts):
            print(f"⌛️현재진행률 {i+1}/{len(posts)}")
            result = await get_blog_article_async(i, post['link'], name)
            if result:
              crawled_results.append(result)
              with open(path, 'w', encoding="utf-8") as f:
                json.dump(crawled_results, f, ensure_ascii=False, indent=2)
                print(f"📁 크롤링 결과 drive에 저장 완료: {json_filename} : index : {i}\n")

          await browser.close()

          # # 3. JSON 파일로 저장 (축제명 기반 파일명)
          # safe_name = name.replace(" ", "_")  # 공백을 밑줄로 바꾸기
          # json_filename = f"results/{safe_name}.json"
          # file_name = f"{safe_name}_블로그_크롤링결과.json"
          # path = '/content/drive/MyDrive/JSON/BLOG/' + file_name


          # with open(path, 'w', encoding="utf-8") as f:
          #     json.dump(crawled_results, f, ensure_ascii=False, indent=2)
          # print(f"\n📁 크롤링 결과 drive에 저장 완료: {json_filename}")

# 🟡 실행
await main()

🔍 "신촌글로벌대학문화축제"로 검색 시작
📦 1번부터 불러오는 중...
📦 101번부터 불러오는 중...
📦 201번부터 불러오는 중...
📦 301번부터 불러오는 중...
📦 401번부터 불러오는 중...
더 이상 결과가 없습니다.

✅ 총 415개 게시글 수집 완료!
✅ 필터링 후 남은 글 수: 401

🔍 블로그 게시글 상세 내용 크롤링 시작...

⌛️현재진행률 1/401
✅ [1] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/신촌글로벌대학문화축제.json : index : 0
⌛️현재진행률 2/401
✅ [2] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/신촌글로벌대학문화축제.json : index : 1
⌛️현재진행률 3/401
✅ [3] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/신촌글로벌대학문화축제.json : index : 2
⌛️현재진행률 4/401
✅ [4] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/신촌글로벌대학문화축제.json : index : 3
⌛️현재진행률 5/401
✅ [5] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/신촌글로벌대학문화축제.json : index : 4
⌛️현재진행률 6/401
✅ [6] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/신촌글로벌대학문화축제.json : index : 5
⌛️현재진행률 7/401
✅ [7] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/신촌글로벌대학문화축제.json : index : 6
⌛️현재진행률 8/401
✅ [8] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/신촌글로벌대학문화축제.json : index : 7
⌛️현재진행률 9/401
🚫 [9] iframe(mainFrame) 못 찾음: https://smartsmpa.tistory.com/9569
⌛️현재진행률 10/401
✅ 

Exception: Browser.close: Connection closed while reading from the driver

In [ ]:
#인덱스 309부터

NameError: name 'crawled_result' is not defined

In [ ]:
async def main():
    # 🎯 크롤링할 축제명 입력
        name = "익산 천만송이 국화축제"

    # 📍 1. 블로그 글 리스트 가져오기
        raw_posts = get_article_list_from_naver_api(name)

    # 제목/요약에 정확히 '더북데이' 들어간 것만 필터링
        posts = filter_relevant_posts(raw_posts, name)


    # 📍 2. 상세 크롤링 시작
        crawled_results = []
        print("\n🔍 블로그 게시글 상세 내용 크롤링 시작...\n")
        async with async_playwright() as p:
        # 브라우저 설정
          browser = await p.chromium.launch(headless=True)
          context = await browser.new_context()
          page = await context.new_page()

          # 3. JSON 파일로 저장 (축제명 기반 파일명) path 설정하기
          safe_name = name.replace(" ", "_")  # 공백을 밑줄로 바꾸기
          json_filename = f"results/{safe_name}.json"
          file_name = f"{safe_name}_블로그_크롤링결과.json"
          path = '/content/drive/MyDrive/JSON/BLOG/' + file_name

          try:
            with open(path, 'r', encoding="utf-8") as f:
              crawled_results = json.load(f)  # 기존 크롤링 데이터를 불러옴
          except FileNotFoundError:
            crawled_results = []  # 파일이 없으면 빈 리스트로 시작


          for i, post in enumerate(posts):
            if i < 243 :
              continue
            if os.path.exists(path):
                with open(path, 'r', encoding='utf-8') as f:
                    existing_data = json.load(f)
            else:
                existing_data = []
            print(f"⌛️현재진행률 {i+1}/{len(posts)}")
            result = await get_blog_article_async(i, post['link'], name)
            if result:
              existing_data.append(result)
              with open(path, 'w', encoding="utf-8") as f:
                json.dump(existing_data, f, ensure_ascii=False, indent=2)
                print(f"\n📁 크롤링 결과 drive에 저장 완료: {json_filename}")

          await browser.close()

          # # 3. JSON 파일로 저장 (축제명 기반 파일명)
          # safe_name = name.replace(" ", "_")  # 공백을 밑줄로 바꾸기
          # json_filename = f"results/{safe_name}.json"
          # file_name = f"{safe_name}_블로그_크롤링결과.json"
          # path = '/content/drive/MyDrive/JSON/BLOG/' + file_name


          # with open(path, 'w', encoding="utf-8") as f:
          #     json.dump(crawled_results, f, ensure_ascii=False, indent=2)
          # print(f"\n📁 크롤링 결과 drive에 저장 완료: {json_filename}")

# 🟡 실행
await main()

ERROR:asyncio:Future exception was never retrieved
future: <Future finished exception=Exception('Connection closed while reading from the driver')>
Exception: Connection closed while reading from the driver


🔍 "익산 천만송이 국화축제"로 검색 시작
📦 1번부터 불러오는 중...
📦 101번부터 불러오는 중...
📦 201번부터 불러오는 중...
📦 301번부터 불러오는 중...
📦 401번부터 불러오는 중...
📦 501번부터 불러오는 중...
📦 601번부터 불러오는 중...
📦 701번부터 불러오는 중...
📦 801번부터 불러오는 중...
📦 901번부터 불러오는 중...

✅ 총 1000개 게시글 수집 완료!
✅ 필터링 후 남은 글 수: 716

🔍 블로그 게시글 상세 내용 크롤링 시작...

⌛️현재진행률 244/716
✅ [244] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/익산_천만송이_국화축제.json
⌛️현재진행률 245/716
✅ [245] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/익산_천만송이_국화축제.json
⌛️현재진행률 246/716
✅ [246] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/익산_천만송이_국화축제.json
⌛️현재진행률 247/716
✅ [247] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/익산_천만송이_국화축제.json
⌛️현재진행률 248/716
✅ [248] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/익산_천만송이_국화축제.json
⌛️현재진행률 249/716
✅ [249] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/익산_천만송이_국화축제.json
⌛️현재진행률 250/716
✅ [250] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/익산_천만송이_국화축제.json
⌛️현재진행률 251/716
✅ [251] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/익산_천만송이_국화축제.json
⌛️현재진행률 252/716
✅ [252] 크롤링 성공

📁 크롤링 결과 drive에 저장 완료: results/익산_천만송이